In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_parquet('d:/BankCustomer/data/bank_v4.parquet')
df = data.copy()

In [3]:
df.shape

(11160, 21)

In [4]:
def select_best_features(df):
    X = df.drop(columns=['deposit'])
    y = df['deposit']
    
    scs = SmartCorrelatedSelection(
        variables=None, method='pearson', 
        threshold=0.4, missing_values='ignore', 
        selection_method='variance', 
        estimator=None, scoring='roc_auc', 
        cv=3, confirm_variables=False
    )
    scs_features = set(scs.fit_transform(X, y).columns)
    
    
    model = LogisticRegression(max_iter=1000)
    rfe = RecursiveFeatureElimination(
        model, scoring='roc_auc', cv=3, 
        threshold=0.01, variables=None, 
        confirm_variables=False
    )
    rfe_features = set(rfe.fit_transform(X, y).columns)
    
    return rfe_features.union(scs_features)   
    

<center><b>Feature Selection</b></center>

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
X = df.drop(columns=['deposit'])
y = df['deposit']

In [7]:
from feature_engine.selection import (
    SmartCorrelatedSelection, RecursiveFeatureElimination
)

In [8]:
scs = SmartCorrelatedSelection(
    variables=None, method='pearson', 
    threshold=0.4, missing_values='ignore', 
    selection_method='variance', 
    estimator=None, scoring='roc_auc', 
    cv=3, confirm_variables=False
)
scs.fit(X, y)

SmartCorrelatedSelection(selection_method='variance', threshold=0.4)

In [9]:
scs.transform(X).columns

Index(['age', 'job', 'education', 'default', 'housing', 'loan', 'contact',
       'day', 'duration', 'campaign', 'pdays',
       'yearly_income_div_number_of_children', 'income_mul_age'],
      dtype='object')

In [10]:
model = LogisticRegression(max_iter=1000)
rfe = RecursiveFeatureElimination(
    model, scoring='roc_auc', cv=3, 
    threshold=0.01, variables=None, 
    confirm_variables=False
)
rfe.fit(X, y)

RecursiveFeatureElimination(estimator=LogisticRegression(max_iter=1000))

In [11]:
rfe.transform(X)

,age,balance,duration
0,59,234300,1042
1,56,4500,1467
2,41,127000,1389
3,55,247600,579
4,54,18400,673
...,...,...,...
11155,35,8000,38
11156,34,-720,273
11157,33,100,257
11158,39,73300,83


In [12]:
data[list(select_best_features(df))].to_parquet('d:/BankCustomer/data/bank_v5.parquet', index=False)